# Rowhouse Discover: Finding the Right Split Path

This notebook demonstrates why finding the correct **split path** is essential for flattening nested JSON into clean, well-structured tables.

## The Problem

When you receive JSON data from an API or event stream, documents often have **different structures** depending on their type. If you try to flatten everything into one table, you end up with:

- Many NULL columns (fields that only exist for certain document types)
- Rows that don't represent a single logical "event"
- Confusion about what each row actually means

## The Solution

Use `rowhouse.discover` to **automatically find the field** that determines document structure (the "splitter"), then use `rowhouse.unfurl` to create **separate, well-defined tables** for each document type.

In [ ]:
import sys
sys.path.insert(0, '..')  # For local development

import pandas as pd
from discover import StructureAnalyzer
from unfurl import JsonProcessor

## Step 1: The Raw Data

Imagine you've received event data from an e-commerce system. There are three types of events, each with a **different structure**:

In [ ]:
# Sample event stream with different document types
events = [
    # ORDER events have items array
    {
        "event": {"type": "OrderPlaced", "id": "evt-001", "timestamp": "2024-01-15T10:00:00Z"},
        "data": {
            "orderId": "ORD-100",
            "customer": {"name": "Alice", "email": "alice@example.com"},
            "items": [
                {"sku": "LAPTOP-1", "name": "MacBook Pro", "price": 2499.00, "qty": 1},
                {"sku": "CASE-1", "name": "Laptop Case", "price": 49.99, "qty": 1}
            ],
            "total": 2548.99
        }
    },
    {
        "event": {"type": "OrderPlaced", "id": "evt-002", "timestamp": "2024-01-15T11:00:00Z"},
        "data": {
            "orderId": "ORD-101",
            "customer": {"name": "Bob", "email": "bob@example.com"},
            "items": [
                {"sku": "PHONE-1", "name": "iPhone 15", "price": 999.00, "qty": 2}
            ],
            "total": 1998.00
        }
    },
    # USER events have preferences array
    {
        "event": {"type": "UserRegistered", "id": "evt-003", "timestamp": "2024-01-15T09:00:00Z"},
        "data": {
            "userId": "USR-200",
            "profile": {"name": "Charlie", "tier": "gold"},
            "preferences": [
                {"category": "electronics", "notify": True},
                {"category": "books", "notify": False}
            ]
        }
    },
    {
        "event": {"type": "UserRegistered", "id": "evt-004", "timestamp": "2024-01-15T09:30:00Z"},
        "data": {
            "userId": "USR-201",
            "profile": {"name": "Diana", "tier": "silver"},
            "preferences": [
                {"category": "clothing", "notify": True}
            ]
        }
    },
    # REVIEW events have flat structure
    {
        "event": {"type": "ReviewPosted", "id": "evt-005", "timestamp": "2024-01-15T14:00:00Z"},
        "data": {
            "reviewId": "REV-300",
            "productSku": "LAPTOP-1",
            "rating": 5,
            "comment": "Amazing laptop!"
        }
    },
    {
        "event": {"type": "ReviewPosted", "id": "evt-006", "timestamp": "2024-01-15T15:00:00Z"},
        "data": {
            "reviewId": "REV-301",
            "productSku": "PHONE-1",
            "rating": 4,
            "comment": "Great phone, battery could be better"
        }
    }
]

print(f"Total events: {len(events)}")
print(f"Event types: {set(e['event']['type'] for e in events)}")

## Step 2: The WRONG Way - One Giant Table

What if we try to flatten ALL events into a single table? We'd need to include every possible field...

In [ ]:
# BAD APPROACH: One config that tries to capture everything
bad_config = {
    "OrderPlaced": {  # We'll process all events as if they were orders
        "table_name": "all_events",
        "fields": [
            # Event metadata (exists in all)
            {"source": "event.type", "alias": "event_type", "type": "string"},
            {"source": "event.id", "alias": "event_id", "type": "string"},
            {"source": "event.timestamp", "alias": "timestamp", "type": "timestamp"},
            
            # Order fields (only in OrderPlaced)
            {"source": "data.orderId", "alias": "order_id", "type": "string"},
            {"source": "data.customer.name", "alias": "customer_name", "type": "string"},
            {"source": "data.items[].sku", "alias": "item_sku", "type": "string"},
            {"source": "data.items[].price", "alias": "item_price", "type": "float"},
            {"source": "data.total", "alias": "order_total", "type": "float"},
            
            # User fields (only in UserRegistered)
            {"source": "data.userId", "alias": "user_id", "type": "string"},
            {"source": "data.profile.name", "alias": "profile_name", "type": "string"},
            {"source": "data.profile.tier", "alias": "tier", "type": "string"},
            {"source": "data.preferences[].category", "alias": "pref_category", "type": "string"},
            
            # Review fields (only in ReviewPosted)
            {"source": "data.reviewId", "alias": "review_id", "type": "string"},
            {"source": "data.rating", "alias": "rating", "type": "integer"},
            {"source": "data.comment", "alias": "comment", "type": "string"},
        ]
    },
    "UserRegistered": {"table_name": "all_events", "fields": []},  # Empty - just to route
    "ReviewPosted": {"table_name": "all_events", "fields": []},
}

# Process only OrderPlaced events to show the problem
processor = JsonProcessor(split_path=['event', 'type'], config=bad_config)
processor.set_file_metadata("events.json", "2024-01-15")

# Filter to just orders for this demo
order_events = [e for e in events if e['event']['type'] == 'OrderPlaced']
result = processor.process_messages(order_events)

print("ONE BIG TABLE (OrderPlaced events only):")
print("=" * 100)
df = result['OrderPlaced']
display_cols = [c for c in df.columns if not c.startswith('s3_')]
df[display_cols]

### Problems with this approach:

1. **Many NULL columns** - User and Review fields are always NULL for Order events
2. **Wasted space** - Every row carries columns it doesn't need
3. **Confusing schema** - What does a row represent? An order? A user? A review?
4. **Array explosion issues** - If we included all event types, rows would multiply incorrectly

---

## Step 3: Use Discover to Find the Split Path

The `StructureAnalyzer` examines your documents and finds the field that best predicts document structure.

In [ ]:
analyzer = StructureAnalyzer()

# Analyze the documents
results = analyzer.find_splitters(events)

print("SPLITTER ANALYSIS")
print("=" * 60)
for r in results:
    marker = " <-- BEST" if r == results[0] else ""
    print(f"\n{r.field}{marker}")
    print(f"  Score: {r.score:.2f}")
    print(f"  Distinct values: {r.distinct_values}")
    print(f"  Coverage: {r.coverage:.0%}")

In [ ]:
# Get the human-readable summary
print(analyzer.describe(events))

### What the Score Means

The score uses **Jaccard similarity** to measure how well a field predicts structure:

- **High score** = Documents with the same value have similar paths, documents with different values have different paths
- `event.type` wins because:
  - All `OrderPlaced` events have the same fields (`data.items[]`, `data.customer`, etc.)
  - All `UserRegistered` events have different fields (`data.preferences[]`, `data.profile`, etc.)
  - The field perfectly predicts which paths exist

---

## Step 4: The RIGHT Way - Separate Tables Per Event Type

Now we create **focused configurations** for each event type, using the discovered split path.

In [ ]:
# Get the best split path
best = results[0]
split_path = best.field.split('.')
print(f"Using split_path: {split_path}")

In [ ]:
# GOOD APPROACH: Separate, focused configs for each event type
good_config = {
    "OrderPlaced": {
        "table_name": "orders",
        "fields": [
            {"source": "event.id", "alias": "event_id", "type": "string"},
            {"source": "event.timestamp", "alias": "timestamp", "type": "timestamp"},
            {"source": "data.orderId", "alias": "order_id", "type": "string"},
            {"source": "data.customer.name", "alias": "customer", "type": "string"},
            {"source": "data.customer.email", "alias": "email", "type": "string"},
            {"source": "data.items[].sku", "alias": "sku", "type": "string"},
            {"source": "data.items[].name", "alias": "product", "type": "string"},
            {"source": "data.items[].price", "alias": "price", "type": "float"},
            {"source": "data.items[].qty", "alias": "quantity", "type": "integer"},
            {"source": "data.total", "alias": "order_total", "type": "float"},
        ]
    },
    "UserRegistered": {
        "table_name": "users",
        "fields": [
            {"source": "event.id", "alias": "event_id", "type": "string"},
            {"source": "event.timestamp", "alias": "timestamp", "type": "timestamp"},
            {"source": "data.userId", "alias": "user_id", "type": "string"},
            {"source": "data.profile.name", "alias": "name", "type": "string"},
            {"source": "data.profile.tier", "alias": "tier", "type": "string"},
            {"source": "data.preferences[].category", "alias": "pref_category", "type": "string"},
            {"source": "data.preferences[].notify", "alias": "pref_notify", "type": "boolean"},
        ]
    },
    "ReviewPosted": {
        "table_name": "reviews",
        "fields": [
            {"source": "event.id", "alias": "event_id", "type": "string"},
            {"source": "event.timestamp", "alias": "timestamp", "type": "timestamp"},
            {"source": "data.reviewId", "alias": "review_id", "type": "string"},
            {"source": "data.productSku", "alias": "product_sku", "type": "string"},
            {"source": "data.rating", "alias": "rating", "type": "integer"},
            {"source": "data.comment", "alias": "comment", "type": "string"},
        ]
    }
}

# Process ALL events with the proper split path
processor = JsonProcessor(split_path=split_path, config=good_config)
processor.set_file_metadata("events.json", "2024-01-15")
result = processor.process_messages(events)

print(f"Created {len(result)} tables from {len(events)} events")

### Orders Table
Each row = one line item in an order

In [ ]:
df = result['OrderPlaced']
display_cols = [c for c in df.columns if not c.startswith('s3_')]
df[display_cols]

### Users Table
Each row = one preference for a user

In [ ]:
df = result['UserRegistered']
display_cols = [c for c in df.columns if not c.startswith('s3_')]
df[display_cols]

### Reviews Table
Each row = one review (no arrays, so 1:1 with source events)

In [ ]:
df = result['ReviewPosted']
display_cols = [c for c in df.columns if not c.startswith('s3_')]
df[display_cols]

---

## Summary: Before vs After

| Approach | Tables | Columns per Table | NULL Values | Row Meaning |
|----------|--------|-------------------|-------------|-------------|
| **Without Split** | 1 | 15+ (all fields) | Many | Unclear |
| **With Split** | 3 | 6-10 (relevant only) | None | Clear |

### Key Benefits of Using Discover:

1. **Clean schemas** - Each table has only the columns it needs
2. **No NULLs** - Every column has meaningful data
3. **Clear semantics** - Each row represents one logical thing
4. **Correct array handling** - Arrays explode within their own table context
5. **Automated discovery** - Don't guess the split path, let the analyzer find it

In [ ]:
# Final summary
print("RESULT SUMMARY")
print("=" * 50)
for name, df in result.items():
    display_cols = [c for c in df.columns if not c.startswith('s3_')]
    print(f"\n{name}:")
    print(f"  Rows: {len(df)}")
    print(f"  Columns: {len(display_cols)}")
    print(f"  Fields: {', '.join(display_cols)}")